async/awaitの使い方をメモる

[参考](https://fisproject.jp/2018/12/coroutines-with-async-and-await/)

yieldによって作られたコルーチンをジェネレータ型のコルーチン。
async/awaitによって作られたコルーチンをネイティブ型のコルーチンと呼ぶ、らしい。

# 基本のパターン

In [33]:
class averager:
    def __await__(self):
        total = 0
        count = 0
        average = None
        while True:
            term = yield
            print(f"  received {term}")
            if term is None:
                break
            total += term
            count += 1
            average = total / count
        return count, average

In [36]:
async def grouper(results, key):    
    while True:
        print(" start waiting")
        results[key] = await averager()
        print(" end waiting")

In [37]:
data = {
    "a": range(2),
    "b": range(3)
}

results = {}
for k in data:
    print(f"start {k}")
    g = grouper(results, k)
    g.send(None) # send()ではダメ
    for v in data[k]:
        print(f"  send {v}")
        g.send(v)
    g.send(None)
    print(f"end {k}")
print(results)

start a
 start waiting
  send 0
  received 0
  send 1
  received 1
  received None
 end waiting
 start waiting
end a
start b
 start waiting
  send 0
  received 0
  send 1
  received 1
  send 2
  received 2
  received None
 end waiting
 start waiting
end b
{'a': (2, 0.5), 'b': (3, 1.0)}


ポイント１：
* `async def`でネイティブコルーチン関数が作られる
* ネイティブコルーチン関数は、`await`の部分で、実行を一時停止できる
* `await`の右側には `awaitable` な値が来る
  * `awaitable`な値とは`__await__`をもつオブジェクト

ポイント２：
* 'end waiting'のあと、再び'start waiting'が始まって、それに対応するend waitingはない
* この動作から想像するに、asyncは、awaitが実行終了の状態になったら即座に終わる、という動作をするポイ
  * 複数ある場合はどうなるの？

ざっくり
* `awaitable`：途中で止まるかもしれない評価式
* `async`：途中でawaitableを呼ぶ関数
という理解で良いのかな・・？

上のawaitの例では、yieldによって処理が止まり、外部からのsendで処理が再開された。普通の非同期プログラミングでは、典型的には外部への問い合わせなどで止まるはず。

# 1つのasync defに複数のawaitがある場合

In [44]:
async def grouper2(r1, r2, key):    
    while True:
        print(" start waiting 1")
        r1[key] = await averager()
        print(" end waiting 1")
        print(" start waiting 2")
        r2[key] = await averager()
        print(" end waiting 2")



data = {
    "a": range(2),
    "b": range(3)
}

r1, r2 = {}, {}
for k in data:
    print(f"start {k}")
    g = grouper2(r1, r2, k)
    g.send(None) # send()ではダメ
    for v in data[k]:
        print(f"  send {v}")
        g.send(v)
    g.send(None)
    print(f"end {k}")
print(r1)
print(r2)

start a
 start waiting 1
  send 0
  received 0
  send 1
  received 1
  received None
 end waiting 1
 start waiting 2
end a
start b
 start waiting 1
  send 0
  received 0
  send 1
  received 1
  send 2
  received 2
  received None
 end waiting 1
 start waiting 2
end b
{'a': (2, 0.5), 'b': (3, 1.0)}
{}


2つ目の`await averager`に来たところでasyncが終わっている。謎の挙動。ないとは思うけど、別のawaitableを当てたら挙動は変わるか・・？

# 1つのasync defに複数のawaitがある場合 その2

In [48]:
class aggregator:
    def __await__(self):
        total = 0
        count = 0
        while True:
            term = yield
            if term is None:
                break
            total += term
            count += 1
        return count, total
    
async def grouper3(r1, r2, key):    
    while True:
        print(" start waiting 1")
        r1[key] = await averager()
        print(" end waiting 1")
        print(" start waiting 2")
        r2[key] = await aggregator()
        print(" end waiting 2")



data = {
    "a": range(2),
    "b": range(3)
}

r1, r2 = {}, {}
for k in data:
    print(f"start {k}")
    g = grouper3(r1, r2, k)
    g.send(None) # send()ではダメ
    for v in data[k]:
        print(f"  send {v}")
        g.send(v)
    g.send(None)
    print(f"end {k}")
print(r1)
print(r2)

start a
 start waiting 1
  send 0
  received 0
  send 1
  received 1
  received None
 end waiting 1
 start waiting 2
end a
start b
 start waiting 1
  send 0
  received 0
  send 1
  received 1
  send 2
  received 2
  received None
 end waiting 1
 start waiting 2
end b
{'a': (2, 0.5), 'b': (3, 1.0)}
{}


結論：変わらない

# grouperのwhileは必要なのか・・？

In [49]:
async def grouper4(r, k):
    r1[k] = await averager()

data = {
    "a": range(2),
    "b": range(3)
}

r1 = {}
for k in data:
    g = grouper4(r1, k)
    g.send(None)
    for v in data[k]:
        g.send(v)
    g.send(None)
print(r1)

  received 0
  received 1
  received None


StopIteration: 

結論：StopIterationがキャッチされずに放出された

# ということは

asyn関数の中では、awaitに出会うと
1. awaitが終わるのを待つ
1. 次のawaitに出会うまで処理を進める
  1. 次のawaitに出会ったら終了する
  1. 出会わない場合はStopIterationを返す

という挙動をするということか。分かりずらい。こうなっている理由が分からない  